# Handling Unstructured Meshes

Unstructured meshes are the most commonly used type of mesh. `MEDCouplingUMesh` is the class responsible for representing these meshes in MEDCoupling. `MEDCouplingUMesh` inherits from the `MEDCouplingPointSet` class. `MEDCouplingPointSet` handles all methods related to coordinates. `MEDCouplingUMesh` has two additional attributes beyond those of `MEDCouplingPointSet` that describe the list of nodes contributing to a cell (i.e., connectivity).

## Objectives

The goal here is to manipulate unstructured meshes (extracting parts, etc.). Several points will be addressed in this exercise:

> - Modifying the coordinates of a mesh
> - Extracting a cross-section of a mesh
> - Extracting a part of the mesh based on cell identifiers
> - Handling indices, etc.
> - Manipulating downward connectivity

<img src="UMesh1.png" style="width:500px;">

## Start of Implementation


In [ ]:
import medcoupling as mc

Build a mesh. This mesh, `mesh3D`, artificially contains two types of cells (`mc.NORM_HEXA8` and `mc.NORM_POLYHED`) to handle the mixing of geometric types. `mesh3D` is an extruded mesh containing 18 cells composed of 3 levels along the Z-axis, with each level having 6 cells. Simply copy and paste the following lines to construct the mesh (the real purpose of the exercise comes later):

In [ ]:
# fmt: off
coords = [
    0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.25, 0.0, 1.0, 0.0, 0.0, 1.0, 1.5, 0.0, 2.0, 0.0, 0.0, 2.0, 1.0, 0.0, 1.0, 2.0, 0.0, 0.0, 2.0, 0.0, 3.0, 1.0, 0.0, 3.0, 2.0, 0.0, 0.0, 1.0, 0.0, 1.0, 3.0,
    0.0, 2.0, 2.0, 0.0, 2.0, 3.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.25, 1.0, 1.0, 0.0, 1.0, 1.0, 1.5, 1.0, 2.0, 0.0, 1.0, 2.0, 1.0, 1.0, 1.0, 2.0, 1.0, 0.0, 2.0, 1.0, 3.0, 1.0, 1.0, 3.0,
    2.0, 1.0, 0.0, 1.0, 1.0, 1.0, 3.0, 1.0, 2.0, 2.0, 1.0, 2.0, 3.0, 1.0, 0.0, 0.0, 2.0, 1.0, 1.0, 2.0, 1.0, 1.25, 2.0, 1.0, 0.0, 2.0, 1.0, 1.5, 2.0, 2.0, 0.0, 2.0, 2.0, 1.0, 2.0, 1.0, 2.0, 2.0,
    0.0, 2.0, 2.0, 3.0, 1.0, 2.0, 3.0, 2.0, 2.0, 0.0, 1.0, 2.0, 1.0, 3.0, 2.0, 2.0, 2.0, 2.0, 2.0, 3.0, 2.0, 0.0, 0.0, 3.0, 1.0, 1.0, 3.0, 1.0, 1.25, 3.0, 1.0, 0.0, 3.0, 1.0, 1.5, 3.0, 2.0, 0.0,
    3.0, 2.0, 1.0, 3.0, 1.0, 2.0, 3.0, 0.0, 2.0, 3.0, 3.0, 1.0, 3.0, 3.0, 2.0, 3.0, 0.0, 1.0, 3.0, 1.0, 3.0, 3.0, 2.0, 2.0, 3.0, 2.0, 3.0, 3.0,
]
conn = [
    0, 11, 1, 3, 15, 26, 16, 18, 1, 2, 4, 7, 13, 6, -1, 1, 16, 21, 6, -1, 6, 21, 28, 13, -1, 13, 7, 22, 28, -1, 7, 4, 19, 22, -1, 4, 2, 17, 19, -1, 2, 1, 16, 17, -1, 16, 21, 28, 22,
    19, 17, 1, 6, 5, 3, 16, 21, 20, 18, 13, 10, 9, 6, 28, 25, 24, 21, 11, 8, 7, 4, 2, 1, -1, 11, 26, 16, 1, -1, 1, 16, 17, 2, -1, 2, 17, 19, 4, -1, 4, 19, 22, 7, -1, 7, 8, 23, 22, -1,
    8, 11, 26, 23, -1, 26, 16, 17, 19, 22, 23, 7, 12, 14, 13, 22, 27, 29, 28, 15, 26, 16, 18, 30, 41, 31, 33, 16, 17, 19, 22, 28, 21, -1, 16, 31, 36, 21, -1, 21, 36, 43, 28, -1, 28,
    22, 37, 43, -1, 22, 19, 34, 37, -1, 19, 17, 32, 34, -1, 17, 16, 31, 32, -1, 31, 36, 43, 37, 34, 32, 16, 21, 20, 18, 31, 36, 35, 33, 28, 25, 24, 21, 43, 40, 39, 36, 26, 23, 22, 19,
    17, 16, -1, 26, 41, 31, 16, -1, 16, 31, 32, 17, -1, 17, 32, 34, 19, -1, 19, 34, 37, 22, -1, 22, 23, 38, 37, -1, 23, 26, 41, 38, -1, 41, 31, 32, 34, 37, 38, 22, 27, 29, 28, 37, 42,
    44, 43, 30, 41, 31, 33, 45, 56, 46, 48, 31, 32, 34, 37, 43, 36, -1, 31, 46, 51, 36, -1, 36, 51, 58, 43, -1, 43, 37, 52, 58, -1, 37, 34, 49, 52, -1, 34, 32, 47, 49, -1, 32, 31, 46,
    47, -1, 46, 51, 58, 52, 49, 47, 31, 36, 35, 33, 46, 51, 50, 48, 43, 40, 39, 36, 58, 55, 54, 51, 41, 38, 37, 34, 32, 31, -1, 41, 56, 46, 31, -1, 31, 46, 47, 32, -1, 32, 47, 49, 34,
    -1, 34, 49, 52, 37, -1, 37, 38, 53, 52, -1, 38, 41, 56, 53, -1, 56, 46, 47, 49, 52, 53, 37, 42, 44, 43, 52, 57, 59, 58,
]
# fmt: on
mesh3D = mc.MEDCouplingUMesh("mesh3D", 3)
mesh3D.allocateCells(18)
mesh3D.insertNextCell(mc.NORM_HEXA8, conn[0:8])
mesh3D.insertNextCell(mc.NORM_POLYHED, conn[8:51])
mesh3D.insertNextCell(mc.NORM_HEXA8, conn[51:59])
mesh3D.insertNextCell(mc.NORM_HEXA8, conn[59:67])
mesh3D.insertNextCell(mc.NORM_POLYHED, conn[67:110])
mesh3D.insertNextCell(mc.NORM_HEXA8, conn[110:118])
mesh3D.insertNextCell(mc.NORM_HEXA8, conn[118:126])
mesh3D.insertNextCell(mc.NORM_POLYHED, conn[126:169])
mesh3D.insertNextCell(mc.NORM_HEXA8, conn[169:177])
mesh3D.insertNextCell(mc.NORM_HEXA8, conn[177:185])
mesh3D.insertNextCell(mc.NORM_POLYHED, conn[185:228])
mesh3D.insertNextCell(mc.NORM_HEXA8, conn[228:236])
mesh3D.insertNextCell(mc.NORM_HEXA8, conn[236:244])
mesh3D.insertNextCell(mc.NORM_POLYHED, conn[244:287])
mesh3D.insertNextCell(mc.NORM_HEXA8, conn[287:295])
mesh3D.insertNextCell(mc.NORM_HEXA8, conn[295:303])
mesh3D.insertNextCell(mc.NORM_POLYHED, conn[303:346])
mesh3D.insertNextCell(mc.NORM_HEXA8, conn[346:354])
myCoords = mc.DataArrayDouble(coords, 60, 3)
myCoords.setInfoOnComponents(["X [m]", "Y [m]", "Z [m]"])
mesh3D.setCoords(myCoords)
mesh3D.orientCorrectlyPolyhedrons()
mesh3D.sortCellsInMEDFileFrmt()
mesh3D.checkConsistencyLight()
renum = mc.DataArrayInt(60)
renum[:15] = list(range(15, 30))
renum[15:30] = list(range(15))
renum[30:45] = list(range(45, 60))
renum[45:] = list(range(30, 45))
mesh3D.renumberNodes(renum, 60)

## Convert Units  

Here, we convert the coordinates from meters to centimeters. It may seem trivial, but it's a very common issue in coupling...

In [ ]:
mesh3D.getCoords()[:] *= 100.0
mesh3D.getCoords().setInfoOnComponents(["X [cm]", "Y [cm]", "Z [cm]"])

<div class="alert alert-block alert-success">  
<b>Note:</b> It is important to update the information on the coordinate components (units) to avoid any ambiguity. The INTERP_KERNEL library includes a unit evaluator.  
</div>  

<div class="alert alert-block alert-success">  
<b>Note:</b> Pay attention to the trick on the first line `[:]` to retrieve the writable version of the coordinates (and not just a temporary copy).  
</div>  

## Find the Different Levels  

The mesh is extruded, making it very regular and aligned with the Ox, Oy, and Oz axes (see figure). We want to determine the Z-levels of the different layers of cubes. Extract the different Z-levels in `mesh3D`, sorted in ascending order. Use the methods `DataArrayDouble.getDifferentValues()` and `DataArrayDouble.sort()`.

In [ ]:
zLev = mesh3D.getCoords()[:, 2]
zLev = zLev.getDifferentValues(1e-12)
zLev.sort()  # In-place sort

## Extract Cell Identifiers  

Extract the 6 cell identifiers of the second row along the Oz axis. There are three possible ways to achieve this. We will explore them from the simplest to the most complex.  

- Using `buildSlice3D()`:  

> A very simple method but CPU-intensive. To find the solution, simply define a plane with a normal vector `[0.,0.,1.]` passing through the point `[0., 0., (zLev[1]+zLev[2])/2]`. The method returns two things: the cut mesh `tmp` (a 2D mesh in terms of mesh-dimension but in a 3D spatial dimension) and, for each 3D surface cell in `tmp`, the identifier of the 3D cell (a volume) that was cut in the original mesh.

In [ ]:
tmp, cellIdsSol1 = mesh3D.buildSlice3D(
    [0.0, 0.0, (zLev[1] + zLev[2]) / 2], [0.0, 0.0, 1.0], 1e-12
)

- **Using the centroids of the cells in `mesh3D`**:  

> Using centroids is a classic technique to identify a set of cells that meet specific geometric criteria. First, compute the centroids of the 3D cells in `mesh3D` using the method `MEDCouplingUMesh.computeCellCenterOfMass()`.  

> Next, select the #2 component of the cell centroids and store the result in `baryZ`. Then, simply select the tuples in `baryZ` that fall within the interval `[zLev[1], zLev[2]]`. The identifiers of these tuples (i.e., their index in `baryZ`) directly correspond to cell identifiers because `computeCellCenterOfMass()` returns an array indexed by cell numbers.

In [ ]:
bary = mesh3D.computeCellCenterOfMass()
baryZ = bary[:, 2]
cellIdsSol2 = baryZ.findIdsInRange(zLev[1], zLev[2])

- **Using `MEDCouplingMappedExtrudedMesh`**:  

> This method relies exclusively on nodal connectivity to determine the extrusion. The coordinates are ignored in this process. To construct a `MEDCouplingMappedExtrudedMesh`, two objects are required: the unstructured 3D mesh, which is actually an extruded mesh, and an unstructured 3D surface mesh (mesh-dim 2) that shares the same coordinates, from which the extrusion will be computed.  

> Start by constructing the 3D surface mesh. To do this, identify the nodes that belong—within a tolerance of `1e-10`—to the plane with normal vector `[0.,0.,1.]` passing through `[0.,0.,zLev[0]]` using `MEDCouplingUMesh.findNodesOnPlane()`. Then, call `MEDCouplingUMesh.buildFacePartOfMySelfNode()` to construct `mesh2D` (refer to the function's documentation for details).

In [ ]:
nodeIds = mesh3D.findNodesOnPlane([0.0, 0.0, zLev[0]], [0.0, 0.0, 1.0], 1e-10)
mesh2D = mesh3D.buildFacePartOfMySelfNode(nodeIds, True)

It is then possible to construct an extruded mesh `extMesh` from `mesh3D` and `mesh2D`. An extruded mesh is created by recognizing an unstructured mesh as the extrusion of a dimension `n-1` mesh (where `n` is the initial dimension of `mesh3D`, which is 3 in this case). If this condition is not met, the construction fails.  

The 2D mesh must be either at the top or bottom of the 3D volumetric mesh, and the last integer specifies the cell from which the 1D wireframe guiding the extrusion will be constructed.

In [ ]:
extMesh = mc.MEDCouplingMappedExtrudedMesh(mesh3D, mesh2D, 0)

We then have the guarantee that, in `extMesh`, the cells are ordered by increasing Z level. It is sufficient to extract the second level using `MEDCouplingMappedExtrudedMesh.getMesh3DIds()`.

In [ ]:
n_cells = mesh2D.getNumberOfCells()
cellIdsSol3 = extMesh.getMesh3DIds()[n_cells : 2 * n_cells]

We then verify that all three solutions are the same.

In [ ]:
print(cellIdsSol1.getValues())
print(cellIdsSol2.getValues())
print(cellIdsSol3.getValues())

## Extract a Subset of a 3D Mesh  

Use the cell identifiers `cellIdsSol2` obtained earlier to extract a subset of `mesh3D`, meaning a mesh containing only a subset of the cells from `mesh3D`.

In [ ]:
mesh3DPart = mesh3D[cellIdsSol2]

<div class="alert alert-block alert-success">
<b>Note:</b> In C++, the underlying method invoked (which is also available in Python) is called `mesh3DPart = mesh3D.buildPartOfMySelf(cellIdsSol2,True)`.</div>

<div class="alert alert-block alert-success">
<b>Note:</b> The geometric type does not matter here at all. The previous instruction takes the cells in the order they are available in the initial mesh.</div>

The object `mesh3DPart` now contains `len(cellIdsSol2)` cells. Cell #0 of `mesh3DPart` corresponds to the cell with identifier `cellIdsSol2[0]` from `mesh3D`, and so on. Thus, `cellIdsSol2` can be seen as a new-to-old mapping array.

At this point, `mesh3DPart` is based on a copy of the coordinate array from `mesh3D`, meaning it still contains 60 nodes, although only 30 are actually used. To remove orphan nodes from `mesh3DPart`, simply invoke `MEDCouplingUMesh.zipCoords()`.

In [ ]:
mesh3DPart.zipCoords()

Now, `mesh3DPart` is based on 30 nodes and contains 6 cells. To be ready for MED file I/O, it is important to check whether `mesh3DPart` is well-ordered, meaning its cells are properly arranged by geometric type. We start by inspecting its current state:

In [ ]:
print(mesh3DPart.advancedRepr())

The following function performs the same task:

In [ ]:
print(mesh3DPart.checkConsecutiveCellTypesAndOrder([mc.NORM_HEXA8, mc.NORM_POLYHED]))

Or :

In [ ]:
print(mesh3DPart.checkConsecutiveCellTypes())

## Extracting Cells Aligned Along a 3D Line

We want to extract from `mesh3D` the 3 cells whose centroids lie along the line defined by `v = [0.,0.,1.]` and passing through `pt = [250.,150.,0.]`. There are two solutions.

- Using the centroids of `mesh3D`: This follows the same principle as above.

In [ ]:
baryXY = bary[:, [0, 1]]
baryXY -= [250.0, 150.0]
magn = baryXY.magnitude()
cellIds2Sol1 = magn.findIdsInRange(0.0, 1e-12)

- Using the extruded mesh `extMesh`: Starting from the single cell in `mesh2D` whose center is at `[250.,150.,0.]`, the method `MEDCouplingMappedExtrudedMesh.getMesh3DIds()` returns the cell IDs row by row.

In [ ]:
bary2 = mesh2D.computeCellCenterOfMass()[:, [0, 1]]
bary2 -= [250.0, 150.0]
magn = bary2.magnitude()
ids = magn.findIdsInRange(0.0, 1e-12)
idStart = int(
    ids
)  # ids is assumed to contain only one value, if not an exception is thrown
ze_range = list(range(idStart, mesh3D.getNumberOfCells(), mesh2D.getNumberOfCells()))
cellIds2Sol2 = extMesh.getMesh3DIds()[ze_range]

Now, we construct this subpart of `mesh3D` using `cellIds2Sol1` or `cellIds2Sol2`.

In [ ]:
mesh3DSlice2 = mesh3D[cellIds2Sol1]
mesh3DSlice2.zipCoords()

## Duplication, Translation, and Aggregation of Meshes  

This part of the exercise is useful for constructing complex meshes or aggregating mesh parts from different processors.  

Here, we aim to duplicate `mesh3DSlice2`, translate it, and aggregate it with the original.  

Perform a full (deep) copy of `mesh3DSlice2` and name it `mesh3DSlice2bis`. Apply a translation of `v=[0.,1000.,0.]` to this copy. Then, merge `mesh3DSlice2` with its translated copy `mesh3DSlice2bis` using `MEDCouplingUMesh.MergeUMeshes()`.

In [ ]:
mesh3DSlice2bis = mesh3DSlice2.deepCopy()
mesh3DSlice2bis.translate([0.0, 1000.0, 0.0])
mesh3DSlice2All = mc.MEDCouplingUMesh.MergeUMeshes([mesh3DSlice2, mesh3DSlice2bis])
mesh3DSlice2All.writeVTK("mesh3DSlice2All.vtu")

<div class="alert alert-block alert-success">
<b>Note:</b> To merge two (or more) unstructured meshes, you need to call `MEDCouplingUMesh.MergeUMeshes()`, then `MEDCouplingUMesh.mergeNodes()` on the result, and finally `MEDCouplingUMesh.zipConnectivityTraducer()`.</div>

## Descending Connectivity  

The goal here is to introduce the concept of descending connectivity.  

Descending connectivity represents the `(n-1)`-dimensional elements that make up each `n`-dimensional cell (where `n` is the mesh dimension, `mesh-dim`). For example, in a 3D mesh where the cells are volumetric (3D), descending connectivity provides the set of faces (2D surfaces) that form the boundaries of these volumes.  

As an example, we aim to extract the internal faces of the `mesh3D` mesh. To achieve this, we need to construct the descending connectivity mesh of `mesh3D` (stored in `mesh3DSurf`). This results in a `(mesh3D.getMeshDimension()-1)`-dimensional mesh, meaning a 2D mesh composed of the faces that bound each of the 3D volume cells of `mesh3D`.  

The method `MEDCoupling.buildDescendingConnectivity()` computes this mesh and simultaneously returns mapping arrays. These arrays establish the correspondence between cell identifiers in `mesh3D` and those in `mesh3DSurf`, and vice versa.  

A face in `mesh3DSurf` is considered internal if and only if it is shared by more than one 3D cell of `mesh3D`. The third and fourth output parameters of the function provide the link between a face and its parent cells (i.e., the volume(s) it delimits). This link is expressed in the indirect index format seen in the first exercise on manipulating the "indirect index" format.

In [ ]:
mesh3DSurf, desc, descIndx, revDesc, revDescIndx = mesh3D.buildDescendingConnectivity()
numberOf3DCellSharing = revDescIndx.deltaShiftIndex()
cellIds = numberOf3DCellSharing.findIdsNotEqual(1)
mesh3DSurfInside = mesh3DSurf[cellIds]
mesh3DSurfInside.writeVTK("mesh3DSurfInside.vtu")

This type of manipulation is very useful for accessing the neighborhood of one or more cells in an unstructured mesh.  

<img src="mesh3DSurfInside.jpg" style="width:500px;">